In [293]:
#!pip install s3fs

In [412]:
sys.path.append('../lib')
import cloudflare_aggregates

create_cloudflare_directory_structure(bucket)

PATH = "../data/"

DATE         = '01-05-2020'
bucket       = os.environ["DATALAKE_PATH"] = 'lakeneto-internal-public'
s3_base_path = os.environ["CLOUDFLARE_FILE_PATH"] = 'processed/platform/cloudflare/'

# Load in initial data sets
df_cloudflare = create_cloudflare_dataframe(bucket, s3_base_path + generate_file_name(DATE, 'csv'))
df_active     = create_active_subscriptions_dataframe('lakeneto-internal-public', 'raw/dataset-active-subscriptions/2020-5-25.csv')
df_idmap      = create_domain_id_map_dataframe(bucket, s3_base_path + 'MerchantIDMap.csv')
df_totals     = create_totals_dataframe(df_cloudflare)

# Join initial data into a MASTER dataset, per active subscription
df_final = create_merchant_dataframe(df_cloudflare, df_active, df_idmap)

# Generate plan and per merchant aggregate dataframes
df_plan_agg         = generate_plan_aggregates(df_final)
df_final            = generate_merchant_aggregates(df_final, df_plan_agg)
df_final            = df_final.set_index('merchant_id')
df_high_utilisation = df_final[df_final['utilisation_rating'] == 2]
    
upload_file(df_final, 'merchant_aggregates')
upload_file(df_plan_agg, 'plan_aggregates')
upload_file(df_totals, 'total_aggregates')
upload_file(df_high_utilisation, 'high_utilisation')

#df_final.to_json('April2020.json', orient='records')